In [9]:
import pandas as pd
from io import StringIO

df = pd.read_csv('data.csv')
df.isnull().sum() #결측치 정보 보기

A    0
B    0
C    1
D    1
dtype: int64

첫번째로 결측값 제거법에 대해 알아보자.

In [10]:
# 누락된 열이 있는 행을 모두 제거한다.
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [11]:
# 해당열에 하나라도 누락된 행이 있으면, 그 행을 제거한다.
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [12]:
df.dropna(how='all') # 모든 열이 공백일때만 행을 삭제한다.
# 이 데이터에선 그런 행이 없음으로 제거가 일어나지 않는다.

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [13]:
# NaN이 아닌 값이 4개 보다 작은 행을 삭제한다. (isnull()애의 결과가 false인 열이 4개 보다 작으면 삭제한다.)
df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [14]:
# 특정 열('C')에 NaN이 있는 행만 삭제한다.
df.dropna(subset=['C'])
# 두번째 행이 제거됐다.

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


결측값을 이번엔 대체하는법을 알아볼것이다.

In [18]:
from sklearn.impute import SimpleImputer
import numpy as np
# 사이킷런의 함수를 이용한 결측치 메꾸기
# NaN값을 결측값으로 취급, 결측값을 그 열의 평균값으로 메꾼다.
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [19]:
# 판다스의 메서드로 결측값 평균으로 메꾸기
df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


데이터 인코딩 방법에 대해서 알아보자

In [131]:
df = pd.DataFrame([
    ['green', 'M', 10.1, 'C2'],
    ['red', 'L',  13.5, 'C1'],
    ['blue', 'XL', 15.3, 'C1'],
])
df.columns = ['color', 'size', 'price', 'classlabel']

순서가 있는 특성 매핑, 예를들어 M < L < XL 와 같이 정량적 또는 개념적인 순서가 있을때, 이 순서에 맞게 인코딩 하기

In [132]:
# 레이블에 순서 있는 특성 매핑 하는법
size_mapping = {
    'XL' : 3,
    'L' : 2,
    'M' : 1}

df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,C2
1,red,2,13.5,C1
2,blue,3,15.3,C1


In [133]:
# 정수값을 원래의 문자표현으로 되돌리는 법
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

순서가 없는 특성에 대해 레이블 인코딩하는법

In [134]:
# 순서 없는 레이블 인코딩 *인코딩된 숫자는 순서가 아니라 그냥 임의값*
class_mapping = {label : idx for idx, label in enumerate(np.unique(df['classlabel']))}
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,0


In [135]:
# 레이블을 다시 원본 문자열로 되돌리는법
inv_class_mapping = {v:k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,C2
1,red,2,13.5,C1
2,blue,3,15.3,C1


In [136]:
# 사이킷런으로 같은 라벨인코딩 하는법
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
df['classlabel'] = y
df

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,0


In [137]:
# 사이킷런으로 라벨인코딩된 데이터셋을 다시 원본 문자열로 되돌리기
df['classlabel'] = class_le.inverse_transform(y)
df

,color,size,price,classlabel
0,green,1,10.1,C2
1,red,2,13.5,C1
2,blue,3,15.3,C1


순서가 없는 특성에 원-핫 인코딩 적용하는법

In [138]:
# 레이블 인코딩 예시
# 아래 코드의 결과로 blue=0, green=1, red=2 의 새로운 color값을 가지게된다.
# 넘파이 의 슬라이스 문법
# X[ :, i] M x N 크기의 행렬에서 i번째 열의 모든 행을 추출한다. 
X = df[['color', 'size', 'price']].values
color_le = LabelEncoder() # 입력데이터는 1차원 데이터를 기대한다.
X[:,0] = color_le.fit_transform(X[:,0]) #즉, 여러 열에 대해서 라벨 인코딩은 번거로울수 있다.
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

위와같은 레이블 인코딩의 문제는 학습알고리즘이 blue < green < red 라는 잘못된 가정을 만들수 있다는것이다. 그럼으로 원-핫 인코딩 기법을 추천한다.

one-hot encoding 이란?
특성에 들어있는 고유한 값마다 더미 특성을 얻게 되는것이다. 예시를 보면 이해가 빠르다.
예시
'color': ['red', 'blue', 'green', 'blue', 'red']
>>> 인코딩 결과
   blue  green  red
0     0      0    1
1     1      0    0
2     0      1    0
3     1      0    0
4     0      0    1

pd.get_dummies( 데이터프레임 ) 메소드 함수를 통해서 할수있다.
dummy_na = False(기본값) : True 일시, 결측값 자체도 별도의 범주로 취급하여 더미변수를 생성한다.
drop_first = False(기본값) : True 일시, 첫번째 더미를 제외하고 생성한다. (사실 n개의 특성을 위해선 n-1개의 더미로 표현이 가능하다. 이를 다중공선성 문제를 피할수있다. ex. 역행렬을 구해야하는 경우)

In [139]:
from sklearn.preprocessing import OneHotEncoder
X = df[['color', 'size', 'price']].values
color_ohe = OneHotEncoder()
color_ohe.fit_transform(X[:, 0].reshape(-1, 1)).toarray()
# reshape(-1, 1) 1차원 벡터를 단일열을 가진 2차원 벡터로 변환해준다. 데이터는 바뀌지 않는다. 머신러닝 라이브러리 입력데이터로 2차원 이상의 배열이 요구되는 경우가 많음으로 이는 중요하다.
# toarray() 는 회소행렬(0이 아닌 요소들만 저장)을 일반 행렬로 변환해준다.

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [140]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder

X = df[['color', 'size', 'price']].values
c_transf = ColumnTransformer([
    ('onehot', OneHotEncoder(dtype =int), [0]), # 첫번째열 'color'에 대해서만 변경 명시
    ('nothing', 'passthrough', [1, 2])          # 2,3번째 열은 변경하지 않고 그대로둔다
])
c_transf.fit_transform(X)
pd.get_dummies(df[['price', 'color', 'size']])  # get_dummies 
df

,color,size,price,classlabel
0,green,1,10.1,C2
1,red,2,13.5,C1
2,blue,3,15.3,C1


그외 함수설정 or lambda함수 - apply를 이용해서 특성에대해 원하는대로 인코딩하기.
예를들어 특정값 이하는 모두 0으로 만들기

In [144]:
df = pd.DataFrame([
    ['green', 'M', 10.1, 'C2'],
    ['red', 'L',  13.5, 'C1'],
    ['blue', 'XL', 15.3, 'C1'],
])
df.columns = ['color', 'size', 'price', 'classlabel']

df['x > M'] = df['size'].apply(lambda x:1 if x in {'L':'XL'} else 0)
df['x > L'] = df['size'].apply(lambda x:1 if x == 'XL' else 0)
del df['size']
df

,color,price,classlabel,x > M,x > L
0,green,10.1,C2,0,0
1,red,13.5,C1,1,0
2,blue,15.3,C1,0,1
